## Exercise 7 - Final

This final exercise combines different parts from the previous exercises.

### Section 1

1 Search Digitraffic's REST interface service (in JSON format) for active trains with the endpoint address `/live-trains` from Helsinki station with the following search parameters:
* arrived_trains=10
* departed_trains=10
* include_nonstopping=False

In [25]:
import requests
import pandas as pd
import sqlite3
import re
url = "https://rata.digitraffic.fi/api/v1/live-trains"
params = {
    "station": "HKI",
    "arrived_trains": 10,
    "departed_trains": 10,
    "include_nonstopping": False
}
response = requests.get(url, params=params)
data = response.json()

data

[{'trainNumber': 70,
  'departureDate': '2025-02-16',
  'operatorUICCode': 10,
  'operatorShortCode': 'vr',
  'trainType': 'IC',
  'trainCategory': 'Long-distance',
  'commuterLineID': '',
  'runningCurrently': False,
  'cancelled': False,
  'version': 290516437812,
  'timetableType': 'REGULAR',
  'timetableAcceptanceDate': '2024-10-31T06:14:29.000Z',
  'timeTableRows': [{'stationShortCode': 'OL',
    'stationUICCode': 370,
    'countryCode': 'FI',
    'type': 'DEPARTURE',
    'trainStopping': True,
    'commercialStop': True,
    'commercialTrack': '3',
    'cancelled': False,
    'scheduledTime': '2025-02-16T10:27:00.000Z',
    'liveEstimateTime': '2025-02-16T10:32:00.000Z',
    'estimateSource': 'LIIKE_USER',
    'actualTime': '2025-02-16T10:37:03.000Z',
    'differenceInMinutes': 10,
    'causes': [{'categoryCode': 'L',
      'detailedCategoryCode': 'L1',
      'thirdCategoryCode': 'L101',
      'categoryCodeId': 352389194,
      'detailedCategoryCodeId': 18153736,
      'thirdCate

2 Read the data into the Pandas dataframe without editing, so that the column headings are as follows:

 `cancelled`,
 `commuterLineID`,
 `departureDate`,
 `operatorShortCode`,
 `operatorUICCode`,
 `runningCurrently`,
 `timeTableRows`,
 `timetableAcceptanceDate`,
 `timetableType`,
 `trainCategory`,
 `trainNumber`,
 `trainType`,
 `version`

In [27]:
columns = [
    "cancelled",
    "commuterLineID",
    "departureDate",
    "operatorShortCode",
    "operatorUICCode",
    "runningCurrently",
    "timeTableRows",
    "timetableAcceptanceDate",
    "timetableType",
    "trainCategory",
    "trainNumber",
    "trainType",
    "version"
]
df = pd.json_normalize(data)
df = df[columns]

3 Delete all other columns except `timeTableRows` and `trainNumber`.

In [28]:
df = df[["timeTableRows", "trainNumber"]]
df

,timeTableRows,trainNumber
0,"[{'stationShortCode': 'OL', 'stationUICCode': ...",70
1,"[{'stationShortCode': 'JY', 'stationUICCode': ...",88
2,"[{'stationShortCode': 'HKI', 'stationUICCode':...",187
3,"[{'stationShortCode': 'TUS', 'stationUICCode':...",974
4,"[{'stationShortCode': 'HNK', 'stationUICCode':...",1630
5,"[{'stationShortCode': 'LPV', 'stationUICCode':...",8202
6,"[{'stationShortCode': 'HKI', 'stationUICCode':...",8209
7,"[{'stationShortCode': 'HKI', 'stationUICCode':...",8385
8,"[{'stationShortCode': 'KKN', 'stationUICCode':...",8500
9,"[{'stationShortCode': 'HKI', 'stationUICCode':...",8505


4 After this, extract the values ​​of the `timeTableRows` column from each train separately (the column value for each train contains several JSON objects) and add `trainNumber`, which identifies the train data.

In [29]:
time_table_rows = []
for index, row in df.iterrows():
    for time_table_row in row["timeTableRows"]:
        time_table_row["trainNumber"] = row["trainNumber"]
        time_table_rows.append(time_table_row)

df_time_table_rows = pd.DataFrame(time_table_rows)

df_time_table_rows

,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialStop,commercialTrack,cancelled,scheduledTime,liveEstimateTime,estimateSource,actualTime,differenceInMinutes,causes,trainReady,trainNumber
0,OL,370,FI,DEPARTURE,True,True,3,False,2025-02-16T10:27:00.000Z,2025-02-16T10:32:00.000Z,LIIKE_USER,2025-02-16T10:37:03.000Z,10.0,"[{'categoryCode': 'L', 'detailedCategoryCode':...","{'source': 'KUPLA', 'accepted': True, 'timesta...",70
1,PKL,819,FI,ARRIVAL,False,NaN,,False,2025-02-16T10:40:00.000Z,NaN,NaN,2025-02-16T10:55:43.000Z,16.0,[],NaN,70
2,PKL,819,FI,DEPARTURE,False,NaN,,False,2025-02-16T10:40:00.000Z,NaN,NaN,2025-02-16T10:55:43.000Z,16.0,[],NaN,70
3,MH,375,FI,ARRIVAL,True,True,1,False,2025-02-16T10:50:00.000Z,NaN,NaN,2025-02-16T11:04:39.000Z,15.0,[],NaN,70
4,MH,375,FI,DEPARTURE,True,True,1,False,2025-02-16T10:51:00.000Z,NaN,NaN,2025-02-16T11:06:58.000Z,16.0,[],NaN,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539,ILR,1030,FI,ARRIVAL,True,True,606,False,2025-02-16T20:15:00.000Z,NaN,NaN,NaN,NaN,[],NaN,60070
1540,ILR,1030,FI,DEPARTURE,True,True,606,False,2025-02-16T19:33:00.000Z,NaN,NaN,2025-02-16T19:34:15.000Z,1.0,[],NaN,60187
1541,PSL,10,FI,ARRIVAL,False,NaN,7,False,2025-02-16T19:43:00.000Z,NaN,NaN,2025-02-16T19:38:06.000Z,-5.0,[],NaN,60187
1542,PSL,10,FI,DEPARTURE,False,NaN,7,False,2025-02-16T19:43:00.000Z,NaN,NaN,2025-02-16T19:38:16.000Z,-5.0,[],NaN,60187


5 Then drop the following columns from the data: `causes`, `differenceInMinutes` and `commercialStop`.

In [30]:
df_time_table_rows.drop(columns=["causes", "differenceInMinutes", "commercialStop"], inplace=True)

6 Fill in the missing values ​​of the `actualTime` column in each row with the values of the `scheduledTime` column.

In [31]:
df_time_table_rows["actualTime"].fillna(df_time_table_rows["scheduledTime"], inplace=True)

df_time_table_rows

C:\Users\jakac\AppData\Local\Temp\ipykernel_6408\1535890779.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_time_table_rows["actualTime"].fillna(df_time_table_rows["scheduledTime"], inplace=True)


,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialTrack,cancelled,scheduledTime,liveEstimateTime,estimateSource,actualTime,trainReady,trainNumber
0,OL,370,FI,DEPARTURE,True,3,False,2025-02-16T10:27:00.000Z,2025-02-16T10:32:00.000Z,LIIKE_USER,2025-02-16T10:37:03.000Z,"{'source': 'KUPLA', 'accepted': True, 'timesta...",70
1,PKL,819,FI,ARRIVAL,False,,False,2025-02-16T10:40:00.000Z,NaN,NaN,2025-02-16T10:55:43.000Z,NaN,70
2,PKL,819,FI,DEPARTURE,False,,False,2025-02-16T10:40:00.000Z,NaN,NaN,2025-02-16T10:55:43.000Z,NaN,70
3,MH,375,FI,ARRIVAL,True,1,False,2025-02-16T10:50:00.000Z,NaN,NaN,2025-02-16T11:04:39.000Z,NaN,70
4,MH,375,FI,DEPARTURE,True,1,False,2025-02-16T10:51:00.000Z,NaN,NaN,2025-02-16T11:06:58.000Z,NaN,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539,ILR,1030,FI,ARRIVAL,True,606,False,2025-02-16T20:15:00.000Z,NaN,NaN,2025-02-16T20:15:00.000Z,NaN,60070
1540,ILR,1030,FI,DEPARTURE,True,606,False,2025-02-16T19:33:00.000Z,NaN,NaN,2025-02-16T19:34:15.000Z,NaN,60187
1541,PSL,10,FI,ARRIVAL,False,7,False,2025-02-16T19:43:00.000Z,NaN,NaN,2025-02-16T19:38:06.000Z,NaN,60187
1542,PSL,10,FI,DEPARTURE,False,7,False,2025-02-16T19:43:00.000Z,NaN,NaN,2025-02-16T19:38:16.000Z,NaN,60187


### Section 2

7 Extract the cell values ​​of the `table` element of the `data.html` file using the regex syntax and connect them to the dataframe created in step 1. **Note**: only regex should be used in the task (E.g. the use of Pandas' read_html() method is prohibited)!

In [32]:
with open("data.html", "r") as file:
    html_content = file.read()

table_data = re.findall(r"<tr.*?>\s*(.*?)\s*</tr>", html_content, re.DOTALL)
table_data = [re.findall(r'<td.*?>\s*(.*?)\s*</td>', row, re.DOTALL) for row in table_data[1:]]

df_html = pd.DataFrame(table_data, columns=[
    "actualTime", "cancelled", "commercialTrack", "countryCode",
    "scheduledTime", "stationShortCode", "stationUICCode",
    "trainStopping", "type", "trainNumber"
])

combined_df = pd.concat([df_time_table_rows, df_html], ignore_index=True)

combined_df

,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialTrack,cancelled,scheduledTime,liveEstimateTime,estimateSource,actualTime,trainReady,trainNumber
0,OL,370,FI,DEPARTURE,True,3,False,2025-02-16T10:27:00.000Z,2025-02-16T10:32:00.000Z,LIIKE_USER,2025-02-16T10:37:03.000Z,"{'source': 'KUPLA', 'accepted': True, 'timesta...",70
1,PKL,819,FI,ARRIVAL,False,,False,2025-02-16T10:40:00.000Z,NaN,NaN,2025-02-16T10:55:43.000Z,NaN,70
2,PKL,819,FI,DEPARTURE,False,,False,2025-02-16T10:40:00.000Z,NaN,NaN,2025-02-16T10:55:43.000Z,NaN,70
3,MH,375,FI,ARRIVAL,True,1,False,2025-02-16T10:50:00.000Z,NaN,NaN,2025-02-16T11:04:39.000Z,NaN,70
4,MH,375,FI,DEPARTURE,True,1,False,2025-02-16T10:51:00.000Z,NaN,NaN,2025-02-16T11:06:58.000Z,NaN,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542,PSL,10,FI,DEPARTURE,False,7,False,2025-02-16T19:43:00.000Z,NaN,NaN,2025-02-16T19:38:16.000Z,NaN,60187
1543,HKI,1,FI,ARRIVAL,True,9,False,2025-02-16T19:48:00.000Z,NaN,NaN,2025-02-16T19:44:06.000Z,NaN,60187
1544,JVS,1272,FI,DEPARTURE,False,,False,2019-08-06T04:48:00.000Z,NaN,NaN,2019-08-06T04:48:00.000Z,NaN,150
1545,R702,98998,FI,ARRIVAL,False,,False,2019-08-06T06:04:39.000Z,NaN,NaN,2019-08-06T06:04:39.000Z,NaN,150


### Section 3

8 Make a SQLite database and insert the dataframe obtained as a result of step 2 to the database.

In [33]:
conn = sqlite3.connect("ex7_sec3.db")

combined_df = combined_df.astype(str)

combined_df.to_sql("train_data", conn, if_exists="replace", index=False)
conn.commit()

query = "SELECT * FROM train_data"
df_from_db = pd.read_sql_query(query, conn)

conn.close()

print(df_from_db.head())

  stationShortCode stationUICCode countryCode       type trainStopping  \
0               OL            370          FI  DEPARTURE          True   
1              PKL            819          FI    ARRIVAL         False   
2              PKL            819          FI  DEPARTURE         False   
3               MH            375          FI    ARRIVAL          True   
4               MH            375          FI  DEPARTURE          True   

  commercialTrack cancelled             scheduledTime  \
0               3     False  2025-02-16T10:27:00.000Z   
1                     False  2025-02-16T10:40:00.000Z   
2                     False  2025-02-16T10:40:00.000Z   
3               1     False  2025-02-16T10:50:00.000Z   
4               1     False  2025-02-16T10:51:00.000Z   

           liveEstimateTime estimateSource                actualTime  \
0  2025-02-16T10:32:00.000Z     LIIKE_USER  2025-02-16T10:37:03.000Z   
1                       nan            nan  2025-02-16T10:55:43.000Z